In [1]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import json

import datasets
import numpy as np
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.adapters import AdapterArguments, AdapterTrainer, AdapterConfigBase, AutoAdapterModel, setup_adapter_training
from transformers import BertTokenizer, BertModelWithHeads, AdapterConfig, EvalPrediction, TextClassificationPipeline
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

transformers.logging.set_verbosity_error()

from torch.utils.data import DataLoader, TensorDataset
import torch

from pdb import set_trace
import transformers.adapters.composition as ac
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),

    'rotten_tomatoes': ("text", None),
    'imdb': ("text", None),
    'yelp_polarity': ("text", None),
    
}

adapter_info = {'cola': {'load_adapter': 'lingaccept/cola@ukp', 'adapter_config': 'pfeiffer'},
                # 'mnli'
                'mrpc': {'load_adapter': 'sts/mrpc@ukp',        'adapter_config': 'pfeiffer'},
                'qnli': {'load_adapter': 'nli/qnli@ukp',        'adapter_config': 'pfeiffer'},
                'qqp' : {'load_adapter': 'sts/qqp@ukp',         'adapter_config': 'pfeiffer'},
                'rte' : {'load_adapter': 'nli/rte@ukp',         'adapter_config': 'pfeiffer'},
                'sst2': {'load_adapter': 'sentiment/sst-2@ukp', 'adapter_config': 'pfeiffer'},
                'stsb': {'load_adapter': 'sts/sts-b@ukp',       'adapter_config': 'pfeiffer'},
                
                'rotten_tomatoes': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-rotten_tomatoes', 'adapter_config': 'pfeiffer'},
                'imdb': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-imdb', 'adapter_config': 'pfeiffer'},
                'yelp_polarity': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-yelp_polarity', 'adapter_config': 'pfeiffer'},
               }

eval_data_dict = {'imdb': 'test', 'yelp_polarity': 'test'}

is_glue = {"cola": True,
            "mnli": True,
            "mrpc": True,
            "qnli": True,
             "qqp": True,
             "rte": True,
            "sst2": True,
            "stsb": True,
            "wnli": True,}

metric_dict = {'rotten_tomatoes': 'sst2', 'imdb': 'sst2', 'yelp_polarity': 'sst2'}

data_dir = './data_ign/'
trial_dir = 'parallel_attack_stealthy_mergeLoss_residual_v1'

In [2]:
epoch = 3

model_name_or_path = 'bert-base-uncased'
pad_to_max_length = True
max_seq_length = 128
output_dir = os.path.join(data_dir, f'evaluate_parallel_loadAdapter/{trial_dir}')

BATCH_SIZE = 128
set_seed(0)

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
)

In [3]:
def get_attacker_path(attacker_name, full_name=None):
    trial_path = os.path.join(data_dir, trial_dir)
    for model_dir in os.listdir(trial_path):
        if full_name:
            if full_name == model_dir:
                return os.path.join(trial_path, model_dir)
        else:
            if attacker_name in model_dir:
                return os.path.join(trial_path, model_dir)
                
def get_epoch_path(attacker_path, attacker_name, epoch=None):
    if epoch == None:
        return os.path.join(attacker_path, f'trained_adapters/{attacker_name}')
    else:
        checkpoint_name_dict = {}
        for dir_name in sorted(os.listdir(attacker_path)):
            if dir_name.startswith('checkpoint'):
                checkpoint_name_dict[dir_name] = int(dir_name.split('-')[-1])
        checkpoint_epoch = sorted(checkpoint_name_dict.items(), key=lambda x: x[1])[epoch-1][0]
        return os.path.join(attacker_path, f'{checkpoint_epoch}/trained_adapters/{attacker_name}')

In [4]:
def get_num_labels(task_name, raw_datasets):
    # Labels
    if task_name is not None:
        is_regression = task_name == "stsb"
        if not is_regression:
            label_list = raw_datasets["train"].features["label"].names
            num_labels = len(label_list)
        else:
            num_labels = 1
    else:
        # Trying to have good defaults here, don't hesitate to tweak to your needs.
        is_regression = raw_datasets["train"].features["label"].dtype in ["float32", "float64"]
        if is_regression:
            num_labels = 1
        else:
            # A useful fast method:
            # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
            label_list = raw_datasets["train"].unique("label")
            label_list.sort()  # Let's sort it for determinism
            num_labels = len(label_list)
    return num_labels, is_regression

def get_data(task_name, raw_datasets):
    sentence1_key, sentence2_key = task_to_keys[task_name]

    if pad_to_max_length:
        padding = "max_length"

    def preprocess_function(examples):    
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    
        # Map labels to IDs (not necessary for GLUE tasks)
        # if label_to_id is not None and "label" in examples:
            # result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        result["label"] = [(l if l != -1 else -1) for l in examples["label"]]
        return result
    
    raw_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        desc="Running tokenizer on dataset",
    )

    return raw_datasets

def get_compute_metrics(task_name, is_regression):
    if task_name in metric_dict:
        metric = evaluate.load("glue", metric_dict[task_name])
    else:
        metric = evaluate.load("glue", task_name)
    
    def compute_metrics(p: EvalPrediction):
        preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
        if task_name is not None:
            result = metric.compute(predictions=preds, references=p.label_ids)
            if len(result) > 1:
                result["combined_score"] = np.mean(list(result.values())).item()
            return result
        elif is_regression:
            return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
        else:
            return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

    return compute_metrics

def do_round(metric_dict):
    result = {}
    for k, v in metric_dict.items():
        result[k] = round(v, 4)
    return result

In [5]:
tasks = ["cola", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb"]

attacker_adapter_list = []
for attacker in tasks:
    for victim in tasks:
        if attacker == victim:
            continue

        attacker_name = f'{attacker}_attack_{victim}'
        attacker_path = get_attacker_path(attacker_name, None)
        attacker_adapter = get_epoch_path(attacker_path, attacker_name, epoch)

        print(f'{attacker}: {attacker_adapter}')
        attacker_adapter_list.append({'attacker_task': attacker, 'attacker_name': attacker_name, 'attacker_adapter': attacker_adapter})

cola: ./data_ign/parallel_attack_stealthy_mergeLoss_residual_v1/cola_attack_mrpc_20231101-032237/checkpoint-201/trained_adapters/cola_attack_mrpc
cola: ./data_ign/parallel_attack_stealthy_mergeLoss_residual_v1/cola_attack_qnli_20231101-110633/checkpoint-2457/trained_adapters/cola_attack_qnli
cola: ./data_ign/parallel_attack_stealthy_mergeLoss_residual_v1/cola_attack_qqp_20231101-174849/checkpoint-8529/trained_adapters/cola_attack_qqp
cola: ./data_ign/parallel_attack_stealthy_mergeLoss_residual_v1/cola_attack_rte_20231102-104731/checkpoint-201/trained_adapters/cola_attack_rte
cola: ./data_ign/parallel_attack_stealthy_mergeLoss_residual_v1/cola_attack_sst2_20231102-150221/checkpoint-1581/trained_adapters/cola_attack_sst2
cola: ./data_ign/parallel_attack_stealthy_mergeLoss_residual_v1/cola_attack_stsb_20231102-203452/checkpoint-201/trained_adapters/cola_attack_stsb
mrpc: ./data_ign/parallel_attack_stealthy_mergeLoss_residual_v1/mrpc_attack_cola_20231031-230334/checkpoint-201/trained_adapt

In [6]:
dataset_dict = {}
for task in tqdm(tasks):
    raw_datasets = load_dataset("glue", task) if task in is_glue else load_dataset(task)
    num_labels, is_regression = get_num_labels(task, raw_datasets)
    dataset = get_data(task, raw_datasets)
    
    eval_dataset = dataset[eval_data_dict[task]] if task in eval_data_dict else dataset['validation']
    compute_metrics = get_compute_metrics(task, is_regression)

    dataset_dict[task] = {'eval_dataset': eval_dataset, 'compute_metrics': compute_metrics}

100%|██████████| 7/7 [00:23<00:00,  3.32s/it]


In [ ]:
result = {}
for attacker_adapter_dict in attacker_adapter_list:
    attacker_task = attacker_adapter_dict['attacker_task']
    attacker_name = attacker_adapter_dict['attacker_name']
    attacker_adapter = attacker_adapter_dict['attacker_adapter']

    result[attacker_name] = {}
    
    for task in tasks:
        if attacker_task == task:
            continue

        task_name_1 = task
        task_name_2 = attacker_task

        load_adapter_1 = adapter_info[task_name_1]['load_adapter']
        adapter_config_1 = AdapterConfigBase.load(adapter_info[task_name_1]['adapter_config'])

        eval_dataset_1 = dataset_dict[task_name_1]['eval_dataset']
        eval_dataset_2 = dataset_dict[task_name_2]['eval_dataset']

        compute_metrics_1 = dataset_dict[task_name_1]['compute_metrics']
        compute_metrics_2 = dataset_dict[task_name_2]['compute_metrics']

        # We use the AutoAdapterModel class here for better adapter support.
        model = AutoAdapterModel.from_pretrained(
            model_name_or_path,
            ignore_mismatched_sizes=False
        )
        
        adapter1 = model.load_adapter(load_adapter_1)
        adapter2 = model.load_adapter(attacker_adapter)
        
        model.active_adapters = ac.Parallel(adapter1, attacker_name)


        # Convert data to tensors and move to device
        input_ids = torch.tensor(eval_dataset_1['input_ids'])
        token_type_ids = torch.tensor(eval_dataset_1['token_type_ids'])
        attention_mask = torch.tensor(eval_dataset_1['attention_mask'])
        labels = torch.tensor(eval_dataset_1['label'])
        
        # Create a DataLoader
        dataset = TensorDataset(input_ids, token_type_ids, attention_mask, labels)
        dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        model = model.to(device)
        model.eval()
        
        all_logits = []
        all_labels = []
        
        with torch.no_grad():
            for batch in dataloader:
                batch = [b_input.to(device) for b_input in batch]
                b_input_ids, b_token_type_ids, b_attention_mask, b_labels = batch
                inputs = {
                    'input_ids': b_input_ids,
                    'token_type_ids': b_token_type_ids,
                    'attention_mask': b_attention_mask
                }
                
                output1, output2 = model(**inputs)
                logits = output1.logits
                all_logits.append(logits.cpu().numpy())
                all_labels.append(b_labels.cpu().numpy())
        
        # Get predictions
        prediction1 = np.concatenate(all_logits, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)
        
        # Evaluate
        compute_results = compute_metrics_1(EvalPrediction(predictions=prediction1, label_ids=all_labels))
        # print(do_round(compute_results))
        
        result1 = compute_results

        # Convert data to tensors and move to device
        input_ids = torch.tensor(eval_dataset_2['input_ids'])
        token_type_ids = torch.tensor(eval_dataset_2['token_type_ids'])
        attention_mask = torch.tensor(eval_dataset_2['attention_mask'])
        labels = torch.tensor(eval_dataset_2['label'])
        
        # Create a DataLoader
        dataset = TensorDataset(input_ids, token_type_ids, attention_mask, labels)
        dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        all_logits = []
        all_labels = []
        
        with torch.no_grad():
            for batch in dataloader:
                batch = [b_input.to(device) for b_input in batch]
                b_input_ids, b_token_type_ids, b_attention_mask, b_labels = batch
                inputs = {
                    'input_ids': b_input_ids,
                    'token_type_ids': b_token_type_ids,
                    'attention_mask': b_attention_mask
                }
                
                output1, output2 = model(**inputs)
                logits = output2.logits
                all_logits.append(logits.cpu().numpy())
                all_labels.append(b_labels.cpu().numpy())
        
        # Get predictions
        prediction1 = np.concatenate(all_logits, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)
        
        # Evaluate
        compute_results = compute_metrics_2(EvalPrediction(predictions=prediction1, label_ids=all_labels))
        # print(do_round(compute_results))
        
        result2 = compute_results

        result[attacker_name][task_name_1] = {'victim_task': result1, 'attacker_task': result2}

        os.makedirs(output_dir, exist_ok=True)

        result_path = os.path.join(output_dir, f'{attacker_name}.json')
        with open(result_path, 'w') as f:
            json.dump(result, f)

        result1_round = do_round(result1)
        result2_round = do_round(result2)
        
        print(f'{attacker_name} {task_name_1} {result1_round} {result2_round}')

cola_attack_mrpc mrpc {'accuracy': 0.6887, 'f1': 0.8146, 'combined_score': 0.7517} {'matthews_correlation': 0.4891}
cola_attack_mrpc qnli {'accuracy': 0.7959} {'matthews_correlation': 0.3944}
cola_attack_mrpc qqp {'accuracy': 0.7933, 'f1': 0.7777, 'combined_score': 0.7855} {'matthews_correlation': 0.2257}
cola_attack_mrpc rte {'accuracy': 0.574} {'matthews_correlation': 0.3636}
cola_attack_mrpc sst2 {'accuracy': 0.8188} {'matthews_correlation': 0.3633}
cola_attack_mrpc stsb {'pearson': 0.7803, 'spearmanr': 0.854, 'combined_score': 0.8172} {'matthews_correlation': 0.4558}
cola_attack_qnli mrpc {'accuracy': 0.6838, 'f1': 0.8122, 'combined_score': 0.748} {'matthews_correlation': 0.5792}
cola_attack_qnli qnli {'accuracy': 0.495} {'matthews_correlation': 0.5687}
cola_attack_qnli qqp {'accuracy': 0.4704, 'f1': 0.5817, 'combined_score': 0.526} {'matthews_correlation': 0.3321}
cola_attack_qnli rte {'accuracy': 0.5271} {'matthews_correlation': 0.5235}
cola_attack_qnli sst2 {'accuracy': 0.7856} 